# Perform SQL query in natural language

In this notebook, we provide an example of using React agent to perform sql query in natural lanugage. 

## Prerequisites

- Follow [READMD.md](https://github.com/modelscope/agentscope) to install AgentScope.
- Prepare a model configuration. AgentScope supports both local deployed model services (CPU or GPU) and third-party services. More details and example model configurations please refer to our [tutorial](https://modelscope.github.io/agentscope/en/tutorial/203-model.html).
- Get familiar with service functions, ServiceFactory module and build-in ReAct agent. The corresponding example can be found in [ReAct agent example](../conversation_with_react_agent/main.ipynb).

## Note

- The example is tested with the following models. For other models, you may need to adjust the prompt.
    - gpt-4 (The model that [DAIL-SQL](https://github.com/BeachWang/DAIL-SQL) is built on)
    - gpt-3.5-turbo   

## Step 1: Initializing model config and SQL database
First, we initalize the agentscope config and load the corresponding model.

In [ ]:
%env HF_ENDPOINT=https://hf-mirror.com

from typing import Callable
import agentscope
from agentscope.models import load_model_by_config_name
agentscope.init(
    model_configs="./configs/model_configs.json",
)
loaded_model = load_model_by_config_name('gpt-3.5-turbo')

Then, we init and provide the corresponding database path in sqlite file format.
Here we generate the sqlite file using the provided sql commands [here](./database/concert_singer/schema.sql). 
You can also use the .sqlite format file directly.

In [ ]:
from sql_utils import create_sqlite_db_from_schema
example_db_id = "concert_singer"
db_schema_path = "./database/concert_singer/schema.sql"
db_sqlite_path = "./database/concert_singer/concert_singer.sqlite"
create_sqlite_db_from_schema(db_schema_path, db_sqlite_path)


We can show a general description of the database schema using the code below.

In [3]:
from loguru import logger
from sql_utils import SQLPrompt
describe_prompt = SQLPrompt().describe_schema(db_sqlite_path)
sql_description = loaded_model([{"role": "assistant", "content": describe_prompt}]).text
logger.chat(sql_description)

The provided database schema is composed of four tables: stadium, singer, concert, and singer_in_concert.

The "stadium" table has the following fields:

- Stadium_ID: An integer field which is the primary key of the table. It's a unique identification for the stadium.
- Location: A text field that specifies the location of the stadium.
- Name: A text field defining the name of the stadium.
- Capacity: An integer field representing the maximum number of people who can be accommodated in the stadium.
- Highest, Lowest, and Average: Integer fields that could represent various statistics related to the stadium, but their exact use isn't clear without more context.

The "singer" table also has several fields:

- Singer_ID: An integer field which is the primary key of this table and provides a unique identification for each singer.
- Name: A text field giving the name of the singer.
- Country: A text field for the country origin of the singer.
- Song_Name: A text field to store the name of 

## Step 2: Setting up tool functions
Then, we define the tools needed for React Agent to perform SQL query. Namely, our agent should be able to generate sql query given natural language and perform sql query to get the result.

- We referenced a third-party Text-to-SQL [DAIL-SQL](https://github.com/BeachWang/DAIL-SQL) to generate Text-to-SQL prompt.
- We use the query_sqlite service function in the agentscope.service module. 

In [4]:
from agentscope.service import ServiceResponse, ServiceExecStatus, ServiceFactory
from sql_utils import DailSQLPromptGenerator, SQLPrompt

def generate_sql_query(question: str, db_path: str) -> ServiceResponse:
    """
    Generate the sql query based on the natural lanugage question from user.
    Args:
        question (`str`):
            The question asked by user in natural language.
        db_path (`str`):
            The path to the .sqlite file.
    """
    prompt_helper = DailSQLPromptGenerator(db_path)
    prepared_prompt = prompt_helper.generate_prompt({"content": question})
    
    def get_response_from_prompt(prompt: dict, model: Callable) -> str:
        """
        Generate response from prompt using LLM
        """
        messages = [{"role": "assistant", "content": prompt}]
        sql = model(messages).text
        sql = " ".join(sql.replace("\n", " ").split())
        sql = sql.strip().split("/*")[0]
        if sql.startswith("SELECT"):
            response = sql + "\n"
        elif sql.startswith(" "):
            response = "SELECT" + sql + "\n"
        else:
            response = "SELECT " + sql + "\n"
        return response

    sql_response = get_response_from_prompt(
        prepared_prompt["prompt"], model=loaded_model
    )

    return ServiceResponse(
        ServiceExecStatus.SUCCESS,
        sql_response
    )

from agentscope.service import query_sqlite


# Use Service Factory to setup tool functions for LLMs
tools = [
    ServiceFactory.get(generate_sql_query, db_path=db_sqlite_path),
    ServiceFactory.get(query_sqlite, database=db_sqlite_path),
]

## Step 3: Create the ReAct agent to help perform SQL queries

Now, we can init the ReAct Agent using the defined tools and interact with the agent.

In [ ]:
from agentscope.agents import ReActAgent
agent = ReActAgent(
    name="assistant",
    model_config_name='gpt-4',
    tools=tools,
    sys_prompt="You are a helpful agent that preform SQL querys base on natual language instructions.",
    verbose=True, # set verbose to True to show the reasoning process
)

## Step 4: Interact with the SQL ReAct Agent using natural lanugage

Now let's see how the agent have response to our questions:

In [6]:
from agentscope.message import Msg
mss = Msg(
    name="user", 
    content="How many singers do we have?", 
    role="user"
)
logger.chat(mss)
sql_query_mss1 = agent(mss)


user: How many singers do we have?
##################### ITER 1, STEP 1: REASONING ######################
assistant: {
    "thought": "I received a question asking how many singers there are. To get this information, I need to generate a SQL query to count the total number of unique singers in the 'singers' table.",
    "speak": "Let me find that out for you.",
    "function": [
        {
            "name": "generate_sql_query",
            "arguments": {
                "question": "How many singers do we have?"
            }
        }
    ]
}
####################### ITER 1, STEP 2: ACTION #######################
>>> Executing function generate_sql_query ...
>>> END 
system: Execution Results:
1. generate_sql_query:
    [EXECUTE STATUS]: SUCCESS
    [EXECUTE RESULT]: SELECT count(*) FROM singer 


##################### ITER 2, STEP 1: REASONING ######################
assistant: {
    "thought": "The SQL query to count the number of singers was successfully generated. I need to execut

In [7]:
mss = Msg(
    name="user", 
    content="Show all countries and the number of singers in each country.", 
    role="user"
)
logger.chat(mss)
sql_query_mss2 = agent(mss)

user: Show all countries and the number of singers in each country.
##################### ITER 1, STEP 1: REASONING ######################
assistant: {
    "thought": "The user wants to see all countries and the number of singers in each country. I'll generate a SQL query to count the singers by country from the relevant table.",
    "speak": "Let me pull up the list of countries and how many singers each one has.",
    "function": [
        {
            "name": "generate_sql_query",
            "arguments": {
                "question": "Show all countries and the number of singers in each country."
            }
        }
    ]
}
####################### ITER 1, STEP 2: ACTION #######################
>>> Executing function generate_sql_query ...
>>> END 
system: Execution Results:
1. generate_sql_query:
    [EXECUTE STATUS]: SUCCESS
    [EXECUTE RESULT]: SELECT ```sql SELECT country, COUNT(*) AS num_singers FROM singer GROUP BY country ``` This SQL query will list each country along 

In [8]:
mss = Msg(
    name="user", 
    content="How many singers have held concert in multiple stadiums? Give me the name of these singers.", 
    role="user"
)
logger.chat(mss)
sql_query_mss3 = agent(mss)

user: How many singers have held concert in multiple stadiums? Give me the name of these singers.
##################### ITER 1, STEP 1: REASONING ######################
assistant: {
    "thought": "To find out how many singers have held concerts in multiple stadiums and their names, I need to generate and execute a SQL query that counts concerts by singer and filters for those with counts greater than 1 in a 'concerts' table, likely involving a JOIN with the 'singers' table.",
    "speak": "I'll look into which singers have graced multiple stadiums with their performances. One moment, please.",
    "function": [
        {
            "name": "generate_sql_query",
            "arguments": {
                "question": "How many singers have held concert in multiple stadiums? Give me the name of these singers."
            }
        }
    ]
}
####################### ITER 1, STEP 2: ACTION #######################
>>> Executing function generate_sql_query ...
>>> END 
system: Execution Res

You can use the following code to build a conversation with the ReAct agent:

``` python
from agentscope.agents import UserAgent

user = UserAgent(name="User")

x = None
while True:
    x = user(x)
    if x.content == "exit":
        break
    x = agent(x)
```

The capabilities of the ReAct agent are indeed very powerful. By modifying prompts and expanding the corresponding usage tools, the ReAct agent can leverage the combination of LLM and tools to accomplish more complex tasks.

We encourage users to delve deeper into exploring the Agent's functionalities and to try out the capabilities of the LLM agent on their own.